Cadiz station

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow as tf
from bayes_opt import BayesianOptimization
from statsmodels.tsa.seasonal import STL
from scipy.fftpack import fft
import warnings

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h1'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)

    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()

    # df['month'] = df.index.month
    # df['weekofyear'] = df.index.isocalendar().week
    # df['year'] = df.index.isocalendar().year
    # df['sin_week'] = np.sin(2 * np.pi * df['weekofyear'] / 52)
    # df['cos_week'] = np.cos(2 * np.pi * df['weekofyear'] / 52)
    # df['EMA_3'] = df[target_col].ewm(span=3, adjust=False).mean()
    # df['EMA_6'] = df[target_col].ewm(span=6, adjust=False).mean()
    # df['correlation_target_month'] = df[target_col].rolling(window=6).corr(df['month'])
    # df['correlation_target_week'] = df[target_col].rolling(window=6).corr(df['weekofyear'])

    # fft_values = fft(df[target_col].dropna().values)
    # fft_real = np.real(fft_values)[:len(df[target_col])]
    # fft_imag = np.imag(fft_values)[:len(df[target_col])]
    # df['fft_real'] = np.concatenate([fft_real, np.nan * np.ones(len(df) - len(fft_real))])
    # df['fft_imag'] = np.concatenate([fft_imag, np.nan * np.ones(len(df) - len(fft_imag))])

    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # LSTM expects 3D input
    return X_scaled, y, scaler

# ============================================
# 3. Build LSTM Model
# ============================================
def build_lstm_model(input_shape, num_classes=4, num_units=64, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(num_units, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_units // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Objective
# ============================================
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(num_units),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }
    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )
    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train & Evaluate Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(best_params['num_units']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    # Training time
    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    # Inference time per sample
    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)

    # Testing time
    start_test = time.time()
    model.evaluate(X_test, y_test, verbose=0)  # Just to trigger testing phase
    testing_time = time.time() - start_test

    # Evaluation
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Evaluation Results =====")
    print(f"Test Accuracy: {acc:.4f}")
    print("Classification Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    # Save & calculate model size
    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)  # MB
    os.remove("temp_model.h5")

    # Trainable Parameters
    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])

    # RAM usage
    process = psutil.Process(os.getpid())
    ram_usage = process.memory_info().rss / (1024 ** 2)  # MB

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f} seconds")
    print(f"Inference Time per Sample: {inference_time:.6f} seconds")
    print(f"Testing Time: {testing_time:.2f} seconds")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Parameters: {total_params:,}")
    print(f"RAM Usage During Inference: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main
# ============================================
def main():
    train_path =  r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\Cadiz_train.csv"
    test_path =  r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\Cadiz_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\nOptimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\nBest Params:", best_params)

    print("\nTraining Final Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 0.9538    | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 0.9538    | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 0.9231    | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 0.9231    | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 0.9385    | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 0.9692    | 22.97     | 0.3185    | 53.09     | 116.6     |
| 7         | 0.9846    | 21.33     | 0.2934    | 53.39     | 117.3     |
| 8         | 0.9692    | 19.18     | 0.4578    | 53.4      | 121.0     |
| 9         | 0.9846    | 21.52     | 0.3409    | 57.65     | 117.8     |
| 10        | 0.9846    | 18.74     | 0.3241    | 55.46     | 115.6     |
| 11        | 0.9692    | 17.88     | 0.4619    | 62.45     | 115.5     |
| 12   


===== Evaluation Results =====
Test Accuracy: 0.9526
Classification Report:
               precision    recall  f1-score   support

           0     0.7857    0.6875    0.7333        16
           1     0.9167    0.9565    0.9362        69
           2     0.9926    0.9854    0.9890       137
           3     0.9000    0.9000    0.9000        10

    accuracy                         0.9526       232
   macro avg     0.8988    0.8824    0.8896       232
weighted avg     0.9518    0.9526    0.9518       232

Confusion Matrix:
 [[ 11   5   0   0]
 [  3  66   0   0]
 [  0   1 135   1]
 [  0   0   1   9]]

===== Model Metrics =====
Training Time: 10.55 seconds
Inference Time per Sample: 0.002050 seconds
Testing Time: 0.09 seconds
Model Size: 0.26 MB
Trainable Parameters: 63,660
RAM Usage During Inference: 963.72 MB


Cordoba station

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow as tf
from bayes_opt import BayesianOptimization
from statsmodels.tsa.seasonal import STL
from scipy.fftpack import fft
import warnings

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h1'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)

    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()

    # df['month'] = df.index.month
    # df['weekofyear'] = df.index.isocalendar().week
    # df['year'] = df.index.isocalendar().year
    # df['sin_week'] = np.sin(2 * np.pi * df['weekofyear'] / 52)
    # df['cos_week'] = np.cos(2 * np.pi * df['weekofyear'] / 52)
    # df['EMA_3'] = df[target_col].ewm(span=3, adjust=False).mean()
    # df['EMA_6'] = df[target_col].ewm(span=6, adjust=False).mean()
    # df['correlation_target_month'] = df[target_col].rolling(window=6).corr(df['month'])
    # df['correlation_target_week'] = df[target_col].rolling(window=6).corr(df['weekofyear'])

    # fft_values = fft(df[target_col].dropna().values)
    # fft_real = np.real(fft_values)[:len(df[target_col])]
    # fft_imag = np.imag(fft_values)[:len(df[target_col])]
    # df['fft_real'] = np.concatenate([fft_real, np.nan * np.ones(len(df) - len(fft_real))])
    # df['fft_imag'] = np.concatenate([fft_imag, np.nan * np.ones(len(df) - len(fft_imag))])

    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # LSTM expects 3D input
    return X_scaled, y, scaler

# ============================================
# 3. Build LSTM Model
# ============================================
def build_lstm_model(input_shape, num_classes=4, num_units=64, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(num_units, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_units // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Objective
# ============================================
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(num_units),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }
    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )
    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train & Evaluate Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(best_params['num_units']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    # Training time
    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    # Inference time per sample
    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)

    # Testing time
    start_test = time.time()
    model.evaluate(X_test, y_test, verbose=0)  # Just to trigger testing phase
    testing_time = time.time() - start_test

    # Evaluation
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Evaluation Results =====")
    print(f"Test Accuracy: {acc:.4f}")
    print("Classification Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    # Save & calculate model size
    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)  # MB
    os.remove("temp_model.h5")

    # Trainable Parameters
    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])

    # RAM usage
    process = psutil.Process(os.getpid())
    ram_usage = process.memory_info().rss / (1024 ** 2)  # MB

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f} seconds")
    print(f"Inference Time per Sample: {inference_time:.6f} seconds")
    print(f"Testing Time: {testing_time:.2f} seconds")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Parameters: {total_params:,}")
    print(f"RAM Usage During Inference: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\Cordoba_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\Cordoba_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\nOptimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\nBest Params:", best_params)

    print("\nTraining Final Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 0.9516    | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 0.9516    | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 0.8871    | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 0.8871    | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 0.9032    | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 0.9677    | 22.97     | 0.3185    | 53.09     | 116.6     |
| 7         | 0.9355    | 23.41     | 0.4111    | 51.92     | 120.5     |
| 8         | 0.9516    | 20.48     | 0.4867    | 55.25     | 116.3     |
| 9         | 0.9355    | 25.29     | 0.2624    | 55.32     | 117.7     |
| 10        | 0.9677    | 20.15     | 0.201     | 51.02     | 117.4     |
| 11        | 0.9677    | 39.95     | 0.2677    | 86.87     | 88.6      |
| 12   


===== Evaluation Results =====
Test Accuracy: 0.8621
Classification Report:
               precision    recall  f1-score   support

           0     0.7500    0.1765    0.2857        17
           1     0.7000    0.9333    0.8000        45
           2     0.9895    0.9307    0.9592       101
           3     0.7333    1.0000    0.8462        11

    accuracy                         0.8621       174
   macro avg     0.7932    0.7601    0.7228       174
weighted avg     0.8750    0.8621    0.8451       174

Confusion Matrix:
 [[ 3 13  0  1]
 [ 1 42  1  1]
 [ 0  5 94  2]
 [ 0  0  0 11]]

===== Model Metrics =====
Training Time: 9.37 seconds
Inference Time per Sample: 0.002373 seconds
Testing Time: 0.09 seconds
Model Size: 0.29 MB
Trainable Parameters: 69,198
RAM Usage During Inference: 1369.27 MB


Jaen Station

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow as tf
from bayes_opt import BayesianOptimization
from statsmodels.tsa.seasonal import STL
from scipy.fftpack import fft
import warnings

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h1'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)

    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()

    df['month'] = df.index.month
    df['weekofyear'] = df.index.isocalendar().week
    df['year'] = df.index.isocalendar().year
    df['sin_week'] = np.sin(2 * np.pi * df['weekofyear'] / 52)
    df['cos_week'] = np.cos(2 * np.pi * df['weekofyear'] / 52)
    df['EMA_3'] = df[target_col].ewm(span=3, adjust=False).mean()
    df['EMA_6'] = df[target_col].ewm(span=6, adjust=False).mean()
    df['correlation_target_month'] = df[target_col].rolling(window=6).corr(df['month'])
    df['correlation_target_week'] = df[target_col].rolling(window=6).corr(df['weekofyear'])

    # fft_values = fft(df[target_col].dropna().values)
    # fft_real = np.real(fft_values)[:len(df[target_col])]
    # fft_imag = np.imag(fft_values)[:len(df[target_col])]
    # df['fft_real'] = np.concatenate([fft_real, np.nan * np.ones(len(df) - len(fft_real))])
    # df['fft_imag'] = np.concatenate([fft_imag, np.nan * np.ones(len(df) - len(fft_imag))])

    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # LSTM expects 3D input
    return X_scaled, y, scaler

# ============================================
# 3. Build LSTM Model
# ============================================
def build_lstm_model(input_shape, num_classes=4, num_units=64, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(num_units, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_units // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Objective
# ============================================
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(num_units),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }
    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )
    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train & Evaluate Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(best_params['num_units']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    # Training time
    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    # Inference time per sample
    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)

    # Testing time
    start_test = time.time()
    model.evaluate(X_test, y_test, verbose=0)  # Just to trigger testing phase
    testing_time = time.time() - start_test

    # Evaluation
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Evaluation Results =====")
    print(f"Test Accuracy: {acc:.4f}")
    print("Classification Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    # Save & calculate model size
    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)  # MB
    os.remove("temp_model.h5")

    # Trainable Parameters
    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])

    # RAM usage
    process = psutil.Process(os.getpid())
    ram_usage = process.memory_info().rss / (1024 ** 2)  # MB

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f} seconds")
    print(f"Inference Time per Sample: {inference_time:.6f} seconds")
    print(f"Testing Time: {testing_time:.2f} seconds")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Parameters: {total_params:,}")
    print(f"RAM Usage During Inference: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\jaen_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\jaen_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\nOptimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\nBest Params:", best_params)

    print("\nTraining Final Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 0.98      | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 0.96      | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 0.96      | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 0.92      | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 0.96      | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 1.0       | 16.26     | 0.4218    | 88.64     | 84.65     |
| 7         | 1.0       | 17.13     | 0.2918    | 99.98     | 115.5     |
| 8         | 1.0       | 19.53     | 0.3371    | 99.03     | 115.6     |
| 9         | 1.0       | 16.51     | 0.1105    | 99.43     | 42.46     |
| 10        | 0.98      | 62.47     | 0.3844    | 99.36     | 127.1     |
| 11        | 1.0       | 16.24     | 0.2625    | 99.93     | 71.03     |
| 12   

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step



===== Evaluation Results =====
Test Accuracy: 0.9348
Classification Report:
               precision    recall  f1-score   support

           0     1.0000    0.2308    0.3750        13
           1     0.8727    0.9600    0.9143        50
           2     0.9837    1.0000    0.9918       121
           3     0.0000    0.0000    0.0000         0

    accuracy                         0.9348       184
   macro avg     0.7141    0.5477    0.5703       184
weighted avg     0.9547    0.9348    0.9272       184

Confusion Matrix:
 [[  3   7   0   3]
 [  0  48   2   0]
 [  0   0 121   0]
 [  0   0   0   0]]

===== Model Metrics =====
Training Time: 11.70 seconds
Inference Time per Sample: 0.002378 seconds
Testing Time: 0.08 seconds
Model Size: 0.18 MB
Trainable Parameters: 41,038
RAM Usage During Inference: 1761.08 MB


Sevilla station

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow as tf
from bayes_opt import BayesianOptimization
from statsmodels.tsa.seasonal import STL
from scipy.fftpack import fft
import warnings

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h1'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)

    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()

    df['month'] = df.index.month
    df['weekofyear'] = df.index.isocalendar().week
    df['year'] = df.index.isocalendar().year
    df['sin_week'] = np.sin(2 * np.pi * df['weekofyear'] / 52)
    df['cos_week'] = np.cos(2 * np.pi * df['weekofyear'] / 52)
    df['EMA_3'] = df[target_col].ewm(span=3, adjust=False).mean()
    df['EMA_6'] = df[target_col].ewm(span=6, adjust=False).mean()
    df['correlation_target_month'] = df[target_col].rolling(window=6).corr(df['month'])
    df['correlation_target_week'] = df[target_col].rolling(window=6).corr(df['weekofyear'])

    # fft_values = fft(df[target_col].dropna().values)
    # fft_real = np.real(fft_values)[:len(df[target_col])]
    # fft_imag = np.imag(fft_values)[:len(df[target_col])]
    # df['fft_real'] = np.concatenate([fft_real, np.nan * np.ones(len(df) - len(fft_real))])
    # df['fft_imag'] = np.concatenate([fft_imag, np.nan * np.ones(len(df) - len(fft_imag))])

    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # LSTM expects 3D input
    return X_scaled, y, scaler

# ============================================
# 3. Build LSTM Model
# ============================================
def build_lstm_model(input_shape, num_classes=4, num_units=64, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(num_units, input_shape=input_shape, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_units // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Objective
# ============================================
def objective_function(num_units, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(num_units),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_units': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }
    optimizer = BayesianOptimization(
        f=lambda num_units, dropout_rate, epochs, batch_size: objective_function(
            num_units, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )
    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train & Evaluate Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_lstm_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_units=int(best_params['num_units']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    # Training time
    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    # Inference time per sample
    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)

    # Testing time
    start_test = time.time()
    model.evaluate(X_test, y_test, verbose=0)  # Just to trigger testing phase
    testing_time = time.time() - start_test

    # Evaluation
    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Evaluation Results =====")
    print(f"Test Accuracy: {acc:.4f}")
    print("Classification Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    # Save & calculate model size
    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)  # MB
    os.remove("temp_model.h5")

    # Trainable Parameters
    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])

    # RAM usage
    process = psutil.Process(os.getpid())
    ram_usage = process.memory_info().rss / (1024 ** 2)  # MB

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f} seconds")
    print(f"Inference Time per Sample: {inference_time:.6f} seconds")
    print(f"Testing Time: {testing_time:.2f} seconds")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Parameters: {total_params:,}")
    print(f"RAM Usage During Inference: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\sevilla_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\sevilla_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\nOptimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\nBest Params:", best_params)

    print("\nTraining Final Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_units |
-------------------------------------------------------------------------
| 1         | 1.0       | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 1.0       | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 0.9857    | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 0.9857    | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 1.0       | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 1.0       | 22.97     | 0.3185    | 53.09     | 116.6     |
| 7         | 1.0       | 22.24     | 0.4959    | 71.26     | 80.54     |
| 8         | 1.0       | 19.19     | 0.136     | 92.37     | 66.74     |
| 9         | 1.0       | 16.28     | 0.4313    | 75.57     | 111.3     |
| 10        | 1.0       | 16.86     | 0.2711    | 88.25     | 36.35     |
| 11        | 1.0       | 18.27     | 0.3674    | 99.38     | 128.0     |
| 12   

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step



===== Evaluation Results =====
Test Accuracy: 0.8879
Classification Report:
               precision    recall  f1-score   support

           0     0.4167    0.5000    0.4545        10
           1     0.9306    0.8816    0.9054        76
           2     0.9752    0.9219    0.9478       128
           3     0.4444    0.8889    0.5926         9

    accuracy                         0.8879       223
   macro avg     0.6917    0.7981    0.7251       223
weighted avg     0.9135    0.8879    0.8969       223

Confusion Matrix:
 [[  5   5   0   0]
 [  7  67   2   0]
 [  0   0 118  10]
 [  0   0   1   8]]

===== Model Metrics =====
Training Time: 16.20 seconds
Inference Time per Sample: 0.002527 seconds
Testing Time: 0.09 seconds
Model Size: 0.20 MB
Trainable Parameters: 45,436
RAM Usage During Inference: 2204.71 MB
